In [ ]:
!pip3 install keras-visualizer
!pip3 install livelossplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached livelossplot-0.5.5-py3-none-any.whl (22 kB)


In [2]:
import numpy as np
import pandas as pd
from google.colab import drive

import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from IPython.display import Image

import tensorflow  as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D, Flatten, Activation  
from keras_visualizer import visualizer 
from keras import models  
from keras import layers 

from livelossplot import PlotLossesKeras

from sklearn.metrics import accuracy_score

KeyboardInterrupt: ignored

In [ ]:
drive.mount('/content/drive')

DATA

In [ ]:
dataset = '/content/drive/MyDrive/Colab Notebooks/dataset/newdataset.csv'
headers = ['age', 	'workclass' , 	'education' ,	'marital.status' ,	'occupation', 	'sex' ,	'hours.per.week' ,	'native.country', 	'income']
df = pd.read_csv(dataset , usecols=headers)
df.head()

fix Null


In [ ]:
# imputer = SimpleImputer(missing_values='?' , strategy='most_frequent')
# imputer = imputer.fit_transform(dataset)
# df = pd.DataFrame(imputer,columns=headers).astype(dataset.dtypes.to_dict())
# df.head()

In [ ]:
df.info()

Label Encode

In [ ]:
le = LabelEncoder()

for columns in df.columns[df.dtypes == 'object']:
  df[columns] = le.fit_transform(df[columns])
df.head()

In [ ]:
x_dataset = df.drop(columns=['income'])
y_dataset = df['income']
Xtrain,Xtest,Ytrain,Ytest = train_test_split(x_dataset,y_dataset,test_size=.33,random_state=1)

In [ ]:
Xtrain

In [ ]:
Ytrain

Scale

In [ ]:
scaler = StandardScaler()

Xtrain_scale = scaler.fit_transform(Xtrain)
Xtest_scale = scaler.fit_transform(Xtest)

In [ ]:
Xtest_scale

**KERAS**

In [ ]:
from threading import active_count
from pyparsing import actions
model = Sequential()

# 1in 2 hid 1out

model.add(Dense(16,activation='relu',input_dim = 8,))
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))

model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
visualizer(model, format='png', view=True)
Image(filename='graph.png')

In [ ]:
from tensorflow.python import metrics
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile( loss='binary_crossentropy',optimizer=opt,metrics=['accuracy','mean_squared_error'])

In [ ]:
plotlosses = PlotLossesKeras()

In [ ]:
# from keras.callbacks import EarlyStopping,ModelCheckpoint
# callback_a = ModelCheckpoint(filepath = 'bestmod.hdf5',monitor='val_loss',save_best_only=True,save_weights_only=True)
# callback_b = EarlyStopping(monitor='val_loss',patience=20,verbose=1)

In [ ]:
# model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/dataset/oldmodel_100epoch.h5')

In [ ]:
plot_moedl = model.fit(Xtrain_scale,Ytrain,epochs=225,validation_split=0.2,verbose=False,batch_size=10,callbacks =[plotlosses])

In [ ]:
np.argmin(model.history['loss'])

In [ ]:
epoc_range = range(1,101)
plt.plot(epoc_range, plot_moedl.history['loss'],label='training LOSS')
plt.plot(epoc_range, plot_moedl.history['val_loss'],label='Validation LOSS')

plt.legend()
plt.show()

In [ ]:
epoc_range = range(1,101)
plt.plot(epoc_range, plot_moedl.history['accuracy'],label='training accuracy')
plt.plot(epoc_range, plot_moedl.history['val_accuracy'],label='Validation accuracy')

plt.legend()
plt.show()

In [ ]:
model.save('newOptimizemodel.h5')


In [ ]:
pred_score = model.predict(Xtest_scale)
print(pred_score)

In [ ]:
y_pred =  np.where(pred_score>0.5,1,0)
accuracy_score(Ytest,y_pred)

In [ ]:
# input = np.array([53,3,0,2,2,1,40,38])
# input = input.reshape(-1,1)
# scl = scaler.transform(input)
model.predict(Xtest_scale)